# 복잡한 표현식 대신 핼퍼 함수를 작성하자

    Python의 간결한 문법을 이용하면 많은 로직을 표현식 한 줄로 쉽게 작성할 수 있다.

    예를 들어 URL에서 쿼리 문자열을 decode 할때 각 쿼리 문자열 파라미터는 정수 값을 표현한다.

In [2]:
from urllib.parse import parse_qs

my_values = parse_qs('red=5&blue=0&green=',
                    keep_blank_values=True)
print(repr(my_values))

# Python2에서는 urllib.parse 대신 urlparse 모듈을 쓰면 됨

{'red': ['5'], 'blue': ['0'], 'green': ['']}


쿼리 문자열 파라미터에 따라 값이 다양하다. 결과 dictionary에 get method를 사용하면 각 상황에 따라 다른 값을 반환할 것이다.

In [3]:
print('Red:     ', my_values.get('red'))
print('Green:   ', my_values.get('green'))
print('Opacity: ', my_values.get('opacity'))

Red:      ['5']
Green:    ['']
Opacity:  None


__파라미터가 없거나 비어있으면 기본 값으로 0__ 을 할당하게 하면 좋다.  
이 로직에 if 문이나 핼퍼 함수를 쓸 필요까지는 없다고 느끼고 작업을 boolean 표현식으로 처리하는 방법을 선택할 수도 있다.

파이썬의 문법 덕분에 boolean 표현식으로도 쉽게 처리가 가능하다. __이때 사용하는 트릭은 빈 문자열, 빈 리스트, 0이 모두 암시적으로 False로 평가되는 점이다.__ 따라서 다음 표현식들의 결과는 처 번째 서브 표현식이 False일 때 or 연산자 뒤에 오는 서브 표현식을 평가한 값이 된다.

In [20]:
# 퀴리 문자열: 'red=5&blue=0&green='
red = my_values.get('red', [''])[0] or 0
green = my_values.get('green', [''])[0] or 0
opacity = my_values.get('opacity', [''])[0] or 0
print('Red:     %r' % red)
print('Green:   %r' % green)
print('Opacity: %r' % opacity)

Red:     '5'
Green:   0
Opacity: 0


__red는 키가 my_values dictionary에 있다.__  
같은 멤버 하나만 있는 리스트이다. 암시적으로 True가 되므로 red는 or 표현식의 첫 번째 부분을 할당받는다.

__green의 경우는 my_values dictionary의 값이 멤버 하나(빈 문자열)만 있는 리스트다.__  
빈 문자열은 암시적으로 False이므로 or 표현식의 결과는 0이 된다.

__opacity의 경우는 my_values dictionary에 값이 존재하지 않는다.__  
get method는 key가 dictionary에 없으면 두 번재 인수를 반환한다. 이때 기본값은 멤버 하나(빈 문자열)만 있는 리스트다. opacity가 dictionary에 없을 경우 green과 동일한 동작을 수행한다.

하지만 위 코드는 읽기 어려우며 필요한 작업을 다 수행하지도 않는다. 모든 파라미터 값이 정수가 되게 해서 수학식에서도 값들을 사용할 수 있게 하려면 내장 함수 int로 처리해서 문자열을 정수 값으로 parsing해야 한다.

In [8]:
red = int(my_values.get('red', [''])[0] or 0)

문장이 잛아서 좋을 수는 있지만 한줄에 모든 코드를 집어넣는 건 큰 의미가 없다.

Python2.5에 추가된 if/else문을 사용하면 코드를 짧게 유지하며 명확한 표현이 가능하다.

In [9]:
red = my_values.get('red', [''])
red = int(red[0]) if red[0] else 0

덜 복잡한 상황에서는 if/else 조건식을 쓰면 코드를 명확하게 이해할 수 있다. 하지만 여러 줄에 걸친 if/else 문을 대체할 정도로 명확하지는 않다.

In [21]:
green = my_values.get('green', [''])
if green[0]:
    green = int(green[0])
else:
    green = 0

이 로직을 반복해서 사용한다면 헬퍼 함수를 사용하는게 좋다.

In [22]:
def get_first_int(values, key, default=0):
    found = values.get(key, [''])
    if found[0]:
        found = int(found[0])
    else:
        found = default
    return found

위 헬퍼 함수를 쓰면 호출 코드를 명확히 할 수 있다.

In [23]:
green = get_first_int(my_values, 'green')

표현식이 복잡해지면 해당 표현식을 작은 조각으로 분할하고 로직을 헬퍼 함수로 옮기는 것을 고려해야 한다.  
무조건 짧은 코드보다는 가독성이 중요하다. 이해하기 어려운 복잡한 표현식에는 함축적인 문법을 사용하면 안 된다.

## 정리

- Python 문법을 이용해 축약한 표현은 이해하기 어렵다.
- 복잡한 표현식은 헬퍼 함수로 넘기는 것이 좋다.
- if/else 표현식을 이용하면 or 나 and 같은 boolean 연산자를 사용할 때보다 가독성이 좋은 코드를 만들 수 있다.